here original data is used to truncate the lists of time series columns and unlike previous attempts, failure columns are kept to be used as categorical columns.

In [2]:
import pandas as pd
import numpy as np

In [3]:
original_data = pd.read_parquet("../../data/test_107_reduced.parquet")
original_data

,doorcyclecounter,doorforce,doorspeed,doorposition,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,cmdoorfriction,cmelectronicage,cmvibration
0,1166,"[-75.1, -75.1, -92.8, -73.5, -51.9, -46.3, -46...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0531, 0.078...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0451, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,121.300003,0.86,0.9
1,1167,"[-75.1, -75.1, -85.0, -92.1, -52.8, -51.6, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.033, 0.0822...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.048, -...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,120.699997,0.85,0.7
2,1168,"[-75.1, -75.1, -87.9, -88.3, -61.2, -51.1, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0612, 0.086...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0468, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,119.800003,0.85,0.9
3,1169,"[-75.1, -75.1, -85.6, -83.4, -57.2, -51.0, -51...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0505, 0.087...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0474, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,122.000000,0.86,0.8
4,1170,"[-75.1, -75.1, -78.5, -91.4, -59.5, -49.7, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0297, 0.073...","[-0.0492, -0.0492, -0.0492, -0.0492, -0.0483, ...",0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,120.400002,0.85,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,6657,"[-75.1, -75.1, -93.3, -59.5, -38.7, -29.9, -29...","[0.0016, 0.0016, 0.0016, 0.0016, 0.054, 0.0706...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0385, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,127.199997,120.000000,0.87,0.9
4876,6658,"[-75.1, -75.1, -90.3, -87.9, -55.8, -47.9, -47...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0462, 0.078...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0401, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,130.899994,111.699997,0.86,1.0
4877,6659,"[-75.1, -75.1, -90.5, -78.1, -52.6, -49.0, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0475, 0.081...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0397, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,129.500000,120.900002,0.89,1.0
4878,6660,"[-75.1, -75.1, -89.8, -82.6, -53.0, -49.3, -49...","[0.0016, 0.0016, 0.0016, 0.0016, 0.0543, 0.082...","[-0.0424, -0.0424, -0.0424, -0.0424, -0.0397, ...",0.0,0.0,False,0.0,0.0,0.0,False,0.0,120.900002,111.500000,0.86,0.8


In [4]:
# removing a row that had unusually high number of time samples
data = original_data[original_data.doorcyclecounter != 3503]

Defining relevant column types

In [5]:
failure_columns = [
	'cd_counter_rollers_installation',
	'ld_counter_rollers_installation',
	'closing_device_failure',
	'misalignment_cd_vs_ld',
	'misalignment_cd_vs_ld_sill_gap',
	'belt_tension',
	'pulley_is_touching_belt',
	'zero_position',
]

environment_columns = [
	'cmcouplerfriction',
	'cmdoorfriction',
	'cmelectronicage',
	'cmvibration',
]

static_state_columns = [
	'doorcyclecounter',
]

time_state_columns = [
	'doorforce',
	'doorspeed',
	'doorposition',
	# 'time',
]

In [6]:
#Data processing and analysis
cat_cols = [
	'cd_counter_rollers_installation',
	'ld_counter_rollers_installation',
	'closing_device_failure',
	'misalignment_cd_vs_ld',
	'misalignment_cd_vs_ld_sill_gap',
	'belt_tension',
	'pulley_is_touching_belt',
	'zero_position',
]

#make num_cols a list of all columns excluding columns that exist in cat_cols
num_cols = [col for col in data.columns if col not in cat_cols]
num_cols

['doorcyclecounter',
 'doorforce',
 'doorspeed',
 'doorposition',
 'cmcouplerfriction',
 'cmdoorfriction',
 'cmelectronicage',
 'cmvibration']

In [7]:
def sample_series(arrays, target_length):
    """
    Sample values from a series of arrays to produce arrays of equal length.
    
    Parameters:
    arrays (list of list of int/float): Series of arrays with different lengths.
    target_length (int): The length of the arrays after sampling.
    
    Returns:
    list of list of int/float: Series of arrays with the target length.
    """
    sampled_series = []

    for array in arrays:
        if len(array) == target_length:
            sampled_series.append(array)
        else:
            indices = np.linspace(0, len(array) - 1, target_length).astype(int)
            sampled_array = [array[i] for i in indices]
            sampled_series.append(sampled_array)
    
    return sampled_series

# Example usage
# arrays = [[1, 2, 3, 4, 5], [10, 20, 30], [100, 200, 300, 400, 500, 600]]
# target_length = 3
# sampled_series = sample_series(arrays, target_length)
# print(sampled_series)

In [8]:
doorforce_series = sample_series(data['doorforce'],20)
doorspeed_series = sample_series(data['doorspeed'],20)
doorposition_series = sample_series(data['doorposition'],20)

# Create new DataFrames with 'doorforce' column
doorforce_df = pd.DataFrame({'doorforce': doorforce_series})
doorspeed_df = pd.DataFrame({'doorspeed': doorspeed_series})
doorposition_df = pd.DataFrame({'doorposition': doorposition_series})

In [9]:
# Convert values into separate columns
doorforce_df = doorforce_df['doorforce'].apply(pd.Series)
doorspeed_df = doorspeed_df['doorspeed'].apply(pd.Series)
doorposition_df = doorposition_df['doorposition'].apply(pd.Series)

# Rename the columns if needed
doorforce_df.columns = [f'doorforce_{i+1}' for i in range(doorforce_df.shape[1])]
doorspeed_df.columns = [f'doorspeed_{i+1}' for i in range(doorspeed_df.shape[1])]
doorposition_df.columns = [f'doorposition_{i+1}' for i in range(doorposition_df.shape[1])]


In [10]:

truncated_data = data.copy()
truncated_data.drop(columns=['doorforce', 'doorspeed', 'doorposition'], inplace=True)
truncated_data


,doorcyclecounter,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,cmdoorfriction,cmelectronicage,cmvibration
0,1166,0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,121.300003,0.86,0.9
1,1167,0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,120.699997,0.85,0.7
2,1168,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,119.800003,0.85,0.9
3,1169,0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,122.000000,0.86,0.8
4,1170,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,120.400002,0.85,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,6657,0.0,0.0,False,0.0,0.0,0.0,False,0.0,127.199997,120.000000,0.87,0.9
4876,6658,0.0,0.0,False,0.0,0.0,0.0,False,0.0,130.899994,111.699997,0.86,1.0
4877,6659,0.0,0.0,False,0.0,0.0,0.0,False,0.0,129.500000,120.900002,0.89,1.0
4878,6660,0.0,0.0,False,0.0,0.0,0.0,False,0.0,120.900002,111.500000,0.86,0.8


In [11]:
train_data = pd.concat([truncated_data, doorforce_df, doorspeed_df, doorposition_df], axis=1)
train_data

,doorcyclecounter,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,...,doorposition_11,doorposition_12,doorposition_13,doorposition_14,doorposition_15,doorposition_16,doorposition_17,doorposition_18,doorposition_19,doorposition_20
0,1166.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,...,0.3666,0.3660,0.3233,0.2335,0.1541,0.0733,0.0074,-0.0269,-0.0501,-0.0492
1,1167.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,...,0.3666,0.3666,0.3072,0.2297,0.1520,0.0709,0.0080,-0.0264,-0.0501,-0.0492
2,1168.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,...,0.3666,0.3666,0.3117,0.2353,0.1562,0.0710,0.0091,-0.0214,-0.0501,-0.0492
3,1169.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,...,0.3666,0.3666,0.3098,0.2301,0.1516,0.0710,0.0083,-0.0246,-0.0501,-0.0495
4,1170.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,...,0.3669,0.3668,0.3041,0.2265,0.1472,0.0675,0.0085,-0.0241,-0.0499,-0.0492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4876,6658.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,130.899994,...,0.3731,0.3731,0.3731,0.2724,0.1278,0.0209,0.0042,-0.0302,-0.0431,-0.0424
4877,6659.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,129.500000,...,0.3731,0.3731,0.3725,0.2813,0.1379,0.0246,0.0059,-0.0273,-0.0431,-0.0424
4878,6660.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,120.900002,...,0.3731,0.3731,0.3729,0.2712,0.1269,0.0203,0.0039,-0.0319,-0.0431,-0.0424
4879,6661.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,115.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
train_data.dropna(inplace=True, axis=0)
train_data.to_parquet("v6-truncated-20ts.parquet")

,doorcyclecounter,cd_counter_rollers_installation,ld_counter_rollers_installation,closing_device_failure,misalignment_cd_vs_ld,misalignment_cd_vs_ld_sill_gap,belt_tension,pulley_is_touching_belt,zero_position,cmcouplerfriction,...,doorposition_11,doorposition_12,doorposition_13,doorposition_14,doorposition_15,doorposition_16,doorposition_17,doorposition_18,doorposition_19,doorposition_20
0,1166.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,154.899994,...,0.3666,0.3660,0.3233,0.2335,0.1541,0.0733,0.0074,-0.0269,-0.0501,-0.0492
1,1167.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,134.399994,...,0.3666,0.3666,0.3072,0.2297,0.1520,0.0709,0.0080,-0.0264,-0.0501,-0.0492
2,1168.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.600006,...,0.3666,0.3666,0.3117,0.2353,0.1562,0.0710,0.0091,-0.0214,-0.0501,-0.0492
3,1169.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,145.800003,...,0.3666,0.3666,0.3098,0.2301,0.1516,0.0710,0.0083,-0.0246,-0.0501,-0.0495
4,1170.0,0.0,0.0,False,1.0,0.0,0.0,False,0.0,146.100006,...,0.3669,0.3668,0.3041,0.2265,0.1472,0.0675,0.0085,-0.0241,-0.0499,-0.0492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4874,6656.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,127.800003,...,0.3731,0.3731,0.3731,0.2689,0.1259,0.0201,0.0041,-0.0314,-0.0431,-0.0424
4875,6657.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,127.199997,...,0.3731,0.3731,0.3729,0.2883,0.1453,0.0264,0.0062,-0.0251,-0.0431,-0.0424
4876,6658.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,130.899994,...,0.3731,0.3731,0.3731,0.2724,0.1278,0.0209,0.0042,-0.0302,-0.0431,-0.0424
4877,6659.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,129.500000,...,0.3731,0.3731,0.3725,0.2813,0.1379,0.0246,0.0059,-0.0273,-0.0431,-0.0424
